<a href="https://colab.research.google.com/github/RakeshIIT/GenAI/blob/main/movie_review_sentiment_lstm_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

In [6]:
data=pd.read_csv(r"/content/movie_data.csv")
data.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [7]:
X, y = (data['review'].values, data['sentiment'].values)

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
tk = Tokenizer(lower = True)
tk.fit_on_texts(X)
X_seq = tk.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=100, padding='post')

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size = 0.25, random_state = 1)

In [10]:
batch_size = 64
X_train1 = X_train[batch_size:]
y_train1 = y_train[batch_size:]
X_valid = X_train[:batch_size]
y_valid = y_train[:batch_size]

In [11]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
vocabulary_size = len(tk.word_counts.keys())+1
max_words = 100
embedding_size = 32


model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(200))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model.fit(X_train1,y_train1,validation_data=(X_valid,y_valid), batch_size=batch_size,epochs=10)

Epoch 1/10
585/585 [==============================] - 63s 96ms/step - loss: 0.4320 - accuracy: 0.7911 - val_loss: 0.2961 - val_accuracy: 0.8594
Epoch 2/10
585/585 [==============================] - 16s 27ms/step - loss: 0.2252 - accuracy: 0.9146 - val_loss: 0.2303 - val_accuracy: 0.8906
Epoch 3/10
585/585 [==============================] - 9s 16ms/step - loss: 0.1432 - accuracy: 0.9491 - val_loss: 0.2629 - val_accuracy: 0.8750
Epoch 4/10
585/585 [==============================] - 8s 14ms/step - loss: 0.0884 - accuracy: 0.9713 - val_loss: 0.4922 - val_accuracy: 0.8438
Epoch 5/10
585/585 [==============================] - 8s 13ms/step - loss: 0.0584 - accuracy: 0.9819 - val_loss: 0.4110 - val_accuracy: 0.8281
Epoch 6/10
585/585 [==============================] - 7s 12ms/step - loss: 0.0462 - accuracy: 0.9861 - val_loss: 0.5235 - val_accuracy: 0.8438
Epoch 7/10
585/585 [==============================] - 6s 11ms/step - loss: 0.0326 - accuracy: 0.9902 - val_loss: 0.6229 - val_accuracy: 0.82

In [26]:
# Prepare reviews for check
df=data.copy()
Check_set = df.review.values
Check_seq = tk.texts_to_sequences(Check_set)
Check_pad = pad_sequences(Check_seq, maxlen = 100, padding = 'post')
# Predict sentiment
check_predict = model.predict(Check_pad, verbose = 0)
# Prepare data frame
check_df = pd.DataFrame(list(zip(df.review.values, df.sentiment.values, check_predict)), columns = ['review','rating','prediction'])
check_df.prediction = [1 if x >= [0.5] else 0 for x in check_df.prediction]
check_df

,review,rating,prediction
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1,1
1,OK... so... I really like Kris Kristofferson a...,0,0
2,"***SPOILER*** Do not read this, if you think a...",0,1
3,hi for all the people who have seen this wonde...,1,1
4,"I recently bought the DVD, forgetting just how...",0,0
...,...,...,...
49995,"OK, lets start with the best. the building. al...",0,0
49996,The British 'heritage film' industry is out of...,0,0
49997,I don't even know where to begin on this one. ...,0,0
49998,Richard Tyler is a little boy who is scared of...,0,0


In [31]:
print(" Accuracy of the lstm model is ",100- (check_df[check_df['rating']!=check_df['prediction']].shape[0])*100/check_df.shape[0])

 Accuracy of the lstm model is  95.83
